In [1]:
## 调试网络维度不一致问题
## 思路：所应每层网络单独调试
## X = torch.randn(128,112,14,14)  # X.shape
## Classifier[0](X) 

In [2]:
import torch 
import numpy as np
import torch.nn as nn 
from nets.mobilenet_v3_decom import decom_MobileNetV3,mobilenet_v3 

In [3]:
Features, Classifier = decom_MobileNetV3() 
X = torch.randn(128,112,14,14)  # X.shape

In [8]:
print(Classifier)
Classifier(X)


Sequential(
  (0): Sequential(
    (13): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): h_swish(
          (sigmoid): h_sigmoid(
            (relu): ReLU6(inplace=True)
          )
        )
        (3): Conv2d(672, 672, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=672, bias=False)
        (4): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fc): Sequential(
            (0): Linear(in_features=672, out_features=168, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=168, out_features=672, bias=True)
            (3): h_sigmoid(
              (relu): ReLU6(inplace=True)
            )
          )
        )
        (6): h_swish(
          (sigmoid):

RuntimeError: mat1 and mat2 shapes cannot be multiplied (122880x1 and 960x1280)

In [18]:
print(Classifier[0])
X0 = Classifier[0](X)
print( 'Classifier[0] output size = ' , X0.shape)

Sequential(
  (13): InvertedResidual(
    (conv): Sequential(
      (0): Conv2d(112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): h_swish(
        (sigmoid): h_sigmoid(
          (relu): ReLU6(inplace=True)
        )
      )
      (3): Conv2d(672, 672, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=672, bias=False)
      (4): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=672, out_features=168, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=168, out_features=672, bias=True)
          (3): h_sigmoid(
            (relu): ReLU6(inplace=True)
          )
        )
      )
      (6): h_swish(
        (sigmoid): h_sigmoid(
          (relu): ReLU6(inplace=True)
        )
      )

In [19]:
print(Classifier[1]) 
X1 = Classifier[1](X0)  
print('Classifier[1] output size = ' , X1.shape)  
 

Sequential(
  (0): Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): h_swish(
    (sigmoid): h_sigmoid(
      (relu): ReLU6(inplace=True)
    )
  )
)
Classifier[1] output size =  torch.Size([128, 960, 7, 7])


In [25]:
print(Classifier[2]) 
X2 = Classifier[2](X1)
print(X2.shape)

AdaptiveAvgPool2d(output_size=(1, 1))
torch.Size([128, 960, 1, 1])


In [30]:
print(Classifier[3])  
X2_1 = X2.view(X2.size(0), -1)
print(X2_1.shape)
X3 = Classifier[3](X2_1)   
print(X2.shape) 

Sequential(
  (0): Linear(in_features=960, out_features=1280, bias=True)
  (1): h_swish(
    (sigmoid): h_sigmoid(
      (relu): ReLU6(inplace=True)
    )
  )
)
torch.Size([128, 960])
torch.Size([128, 960, 1, 1])


In [31]:
Features, Classifier = decom_MobileNetV3() 
X = torch.randn(128,112,14,14)  # X.shape


print( Classifier[:3] )


Sequential(
  (0): Sequential(
    (13): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): h_swish(
          (sigmoid): h_sigmoid(
            (relu): ReLU6(inplace=True)
          )
        )
        (3): Conv2d(672, 672, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=672, bias=False)
        (4): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fc): Sequential(
            (0): Linear(in_features=672, out_features=168, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=168, out_features=672, bias=True)
            (3): h_sigmoid(
              (relu): ReLU6(inplace=True)
            )
          )
        )
        (6): h_swish(
          (sigmoid):

In [33]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args
    def forward(self, x):
        return x.view((x.size(0),-1))

In [56]:
list_Classifier =Classifier 

C1 = list([list_Classifier[:3], Reshape(), list_Classifier[3:]])

C2 = nn.Sequential(*C1)  
print(C2)
C2(X)


Sequential(
  (0): Sequential(
    (0): Sequential(
      (13): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): h_swish(
            (sigmoid): h_sigmoid(
              (relu): ReLU6(inplace=True)
            )
          )
          (3): Conv2d(672, 672, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), groups=672, bias=False)
          (4): BatchNorm2d(672, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): SELayer(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (fc): Sequential(
              (0): Linear(in_features=672, out_features=168, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=168, out_features=672, bias=True)
              (3): h_sigmoid(
                (relu): ReLU6(inplace=True)
              )
   

tensor([[-0.0594,  0.0050,  0.0143,  ...,  0.0258, -0.0416,  0.0309],
        [-0.0389, -0.0078,  0.0168,  ...,  0.0010, -0.0529,  0.0430],
        [-0.0752, -0.0175,  0.0162,  ...,  0.0177, -0.0149,  0.0304],
        ...,
        [-0.0518,  0.0131,  0.0361,  ...,  0.0469, -0.0340,  0.0161],
        [-0.0667, -0.0059, -0.0048,  ...,  0.0205, -0.0258,  0.0128],
        [-0.0551, -0.0017,  0.0229,  ...,  0.0282, -0.0210,  0.0329]],
       grad_fn=<MulBackward0>)

In [2]:
import torch 
import numpy as np
import torch.nn as nn 
from nets.mobilenet_v3_decom import decom_MobileNetV3,mobilenet_v3 

Features, Classifier = decom_MobileNetV3() 
X = torch.randn(128,112,14,14)  # X.shape
Classifier(X).shape


torch.Size([128, 1280])